<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/dev/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code (Frontend and IPFS Connection)

In [9]:
!killall -9 ipfs

In [21]:
#@title Install IPFS

!pip install coolname gdown papermill
#!pip install asyncio nest-asyncio coolname aioipfs
#!pip install ipfshttpclient
#!npm install -g nodemon
#!npm install -g voodoohop/pollinations/app


%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout dev 
!git pull
!npm install
!npm install -g
%cd -
#!npm install forever -g

/content
fatal: destination path 'pollinations' already exists and is not an empty directory.
/content/pollinations/app
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 8 (delta 6), reused 8 (delta 6), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/voodoohop/pollinations
   9404c2a..5e85a3d  dev        -> origin/dev
Updating 9404c2a..5e85a3d
Fast-forward
 app/src/backend/ipfsWatch.js     | 2 +-
 app/src/network/ipfsConnector.js | 5 +----
 2 files changed, 2 insertions(+), 5 deletions(-)
Already on 'dev'
Your branch is up to date with 'origin/dev'.
Already up to date.
npm WARN @babel/plugin-bugfix-v8-spread-parameters-in-optional-chaining@7.13.12 requires a peer of @babel/core@^7.13.0 but none is installed. You must install peer dependencies yourself.
npm WARN tsutils@3.21.0 requires a peer of typescript@>=2.8.0 || >= 3.2.0-dev || >= 3.3.0-dev || >= 3.4.0-dev || >= 3

In [22]:
#@title Config
from coolname import generate_slug
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
IPFS_PEER = "ipfs.pollinations.ai"
ipfs_root = "/content/ipfs_root"

print(f"---Settings---\nIPFS_ROOT: {ipfs_root}\nIPFS_PEER: {IPFS_PEER}")

---Settings---
IPFS_ROOT: /content/ipfs_root
IPFS_PEER: ipfs.pollinations.ai


In [7]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!echo "Created IPFS resource" > $ipfs_root/output/log

parameters={"bla":2}
for key, default in parameters.items():
    print(default, file=open(f"{ipfs_root}/input/{key}", 'a'))



rm: cannot remove '/content/ipfs_root': No such file or directory


In [9]:
!wget https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
!tar -xvzf go-ipfs_v0.8.0_linux-amd64.tar.gz
%cd go-ipfs
!sudo bash install.sh
%cd -
!ipfs init --profile test
#!mkdir -p /ipfs
#!mkdir -p /ipns
!ipfs config Addresses.API /ip4/0.0.0.0/tcp/5001
!ipfs config --json API.HTTPHeaders.Access-Control-Allow-Origin '["*"]'
!nohup ipfs daemon --enable-namesys-pubsub --enable-pubsub-experiment  &
!sleep 15
!ipfs swarm connect /ip4/18.157.205.205/tcp/4001/p2p/12D3KooWDwQ1R9ZmDRv8aWL4dJ4svS9AYvwfJicQ1F45W5aHAVmy



--2021-05-24 08:44:20--  https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
Resolving dist.ipfs.io (dist.ipfs.io)... 209.94.78.1, 2602:fea2:3::1
Connecting to dist.ipfs.io (dist.ipfs.io)|209.94.78.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25904633 (25M) [application/gzip]
Saving to: ‘go-ipfs_v0.8.0_linux-amd64.tar.gz.1’

go-ipfs_v0.8.0_linu 100%[===================>]  24.70M  10.7MB/s    in 2.3s    

2021-05-24 08:44:23 (10.7 MB/s) - ‘go-ipfs_v0.8.0_linux-amd64.tar.gz.1’ saved [25904633/25904633]

go-ipfs/install.sh
go-ipfs/ipfs
go-ipfs/LICENSE
go-ipfs/LICENSE-APACHE
go-ipfs/LICENSE-MIT
go-ipfs/README.md
/content/go-ipfs
Moved ./ipfs to /usr/local/bin
/content
Error: ipfs daemon is running. please stop it to run this command
Use 'ipfs init --help' for information about this command
nohup: appending output to 'nohup.out'
connect 12D3KooWDwQ1R9ZmDRv8aWL4dJ4svS9AYvwfJicQ1F45W5aHAVmy success


In [23]:
from os.path import join


notebook_path = join(ipfs_root,"latentvisions.ipynb")
notebook_params_path = join(ipfs_root,"parameters.json")

!gdown --id 1zOA0bUJ68zuZXcbCoN2LPNjnkvxzFpv_ -O $notebook_path

!python pollinations/pollinations/get_params.py $notebook_path > $notebook_params_path

Downloading...
From: https://drive.google.com/uc?id=1zOA0bUJ68zuZXcbCoN2LPNjnkvxzFpv_
To: /content/ipfs_root/latentvisions.ipynb
100% 20.2k/20.2k [00:00<00:00, 8.75MB/s]


In [33]:
!echo $ipfs_root
content_id=!ipfs add -r -Q $ipfs_root
content_id=content_id[0]
node_id=!ipfs name publish -Q --offline /ipfs/$content_id
node_id=node_id[0]

#!ipfs pubsub pub $node_id $content_id
print(content_id,node_id)

/content/ipfs_root


KeyboardInterrupt: ignored

In [27]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `http://localhost:3000/incolab.html?node={node_id},{content_id}`

  document.querySelector("#output-area").appendChild(iframe)
'''))



<IPython.core.display.Javascript object>

In [32]:
!ipfs pubsub sub $node_id | DEBUG=* pollinate -p /content/ipfs_root --ipns --receive

  ipfsConnector Connecting to IPFS http://ipfs.pollinations.ai:5001 +0ms
  ipfsConnector NodeID Promise { <pending> } +144ms
  ipfsWatch CLI options {
  path: '/content/ipfs_root',
  receive: true,
  send: false,
  once: false,
  ipns: true
} +0ms
QmRiC6nzaUxQiYGd3eTZhHPjVG8oY2W2sPf9j5aNq6eADt  ipfsWatch Processing remote CID QmRiC6nzaUxQiYGd3eTZhHPjVG8oY2W2sPf9j5aNq6eADt +37s
  ipfsState Getting state for CID QmRiC6nzaUxQiYGd3eTZhHPjVG8oY2W2sPf9j5aNq6eADt +0ms
  ipfsState:_getIPFSState(/) Getting state for dir root QmRiC6nzaUxQiYGd3eTZhHPjVG8oY2W2sPf9j5aNq6eADt +0ms
  utils:ipfsls --- Calling  ipfsls with input QmRiC6nzaUxQiYGd3eTZhHPjVG8oY2W2sPf9j5aNq6eADt +0ms
  utils:ipfsls --- In: QmRiC6nzaUxQiYGd3eTZhHPjVG8oY2W2sPf9j5aNq6eADt +0ms
  utils:ipfsls --- Out: ipfsls : [
  {
    name: 'README.md',
    path: 'QmRiC6nzaUxQiYGd3eTZhHPjVG8oY2W2sPf9j5aNq6eADt/README.md',
    cid: 'QmToji6Yiar29MS8scqikSVzC3d444iiptRjFpSCFSiANe',
    type: 'file'
  },
  {
    name: 'input',
    path: 'QmRiC6

In [ ]:
async def update_ipfs_content(contentid):
  global ipfs_contentid
  contentid = contentid.decode("utf-8")
  
  await client.get(contentid)
  !cp -rv $contentid/* $ipfs_root
  #!rm -r $contentid
  ipfs_contentid = contentid

content_id=!ipfs name resolve k51qzi5uqu5dh0esiimwcc251a5oq9ivhjppf1zqip1cpdjcvtwj1olbdmqk9k
#on_frontend_data(update_ipfs_content)
print(content_id)

['/ipfs/QmQ26epDtE4Bi1TJdUyeEy7F4tA6JGGM7bWycbMq1sp6QP']


In [ ]:
from glob import glob
from os.path import basename
for path in glob(f"{ipfs_root}/input/*"):
  key = basename(path)
  with open(path,'r') as f:
    value = f.read()
    parameters[key] = value

print("New parameters", parameters)

## DISABLED! CODE (DEEP-DAZE)

In [ ]:
#!pip install --upgrade pip

#!pip install --upgrade ipython
#import papermill as pm
asda
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

#parameters["text"] = "A black and white photo of Saeko with her new Elektron FM synthesizer"
def run(parameters, watch_media_paths="./*.jpg"):
  print("Running",parameters)
 
  for filepath in glob(watch_media_paths):
    print("Deleting",filepath,"before starting.")
    os.remove(filepath)

  _params = [["-p", key, value] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]
  #%run latentvisions.ipynb
  #!papermill "./latentvisions.ipynb" ./output 
  cmd = ["papermill", "/content/latentvisions.ipynb","/content/latentvisions_output.ipynb"] + params

  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = False)

  found_media_files=[]

  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    with open(f"{ipfs_root}/log","w") as log_out:
      log_out.write(line.decode("utf-8"))
      log_out.flush()
      
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    #print("new contentid", client.add(ipfs_root, recursive=True))
    
    #continue
    new_found_media_files = glob(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root/output
          print("Adding:", added,"to ipfs root")
          pin=True
    add_and_publish()    
        
while True:
  run(parameters,"./taming_transformers/*.jpg")

NameError: ignored